In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 20 20:40:23 2019
@author: santonas
"""


#%%
import tensorflow as tf
print(tf.__version__)
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from time import sleep

#%%
targets_dict = {
    0:"Thumb Down",
    1:"Stop Sign",
    2:"Sliding Two Fingers Left",
    3:"Sliding Two Fingers Right",
    4:"No gesture",
    5:"Thumb Up"
    }
#%%
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

#%%
def normaliz_data(np_data):
    # Normalisation
    scaler = StandardScaler()
    #scaled_images  = normaliz_data2(np_data)
    scaled_images  = np_data.reshape(-1, 30, 64, 64, 1)
    return scaled_images

#%%
def normaliz_data2(v):
    normalized_v = v / np.sqrt(np.sum(v**2))
    return normalized_v


class Conv3DModel(tf.keras.Model):
    def __init__(self):
        super(Conv3DModel, self).__init__()
        # Convolutions
        self.conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
        self.conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')

        # LSTM & Flatten
        self.convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
        self.flatten =  tf.keras.layers.Flatten(name="flatten")

        # Dense layers
        self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
        self.out = tf.keras.layers.Dense(6, activation='softmax', name="output")
    

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.convLSTM(x)
        #x = self.pool2(x)
        #x = self.conv3(x)
        #x = self.pool3(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.out(x)
#%%
new_model = Conv3DModel()
#%%
new_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.RMSprop())

#%%
new_model.load_weights('weights/3d-cnn-basic')



/home/deepanshu/anaconda3/envs/tf_14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/deepanshu/anaconda3/envs/tf_14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/deepanshu/anaconda3/envs/tf_14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/deepanshu/anaconda3/envs/tf_14/lib

1.14.0
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [2]:
#%%
to_predict = []
num_frames = 0
cap = cv2.VideoCapture(0)
classe =''

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    to_predict.append(cv2.resize(gray, (64, 64)))
    
         
    if len(to_predict) == 30:
        frame_to_predict = np.array(to_predict, dtype=np.float32)
        frame_to_predict = normaliz_data(frame_to_predict)
        #print(frame_to_predict)
        predict = new_model.predict(frame_to_predict)
        classe = targets_dict[np.argmax(predict)]
        
        print('Classe = ',classe, 'Precision = ', np.amax(predict)*100,'%')


        #print(frame_to_predict)
        to_predict = []
        #sleep(0.1) # Time in seconds
        #font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, classe, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0),1,cv2.LINE_AA)


    # Display the resulting frame
    cv2.imshow('Hand Gesture Recognition',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Classe =  Thumb Down Precision =  89.41754102706909 %
Classe =  No gesture Precision =  98.72422218322754 %
Classe =  Thumb Down Precision =  72.92562127113342 %
Classe =  No gesture Precision =  71.40315771102905 %
Classe =  Thumb Down Precision =  98.91197681427002 %
Classe =  Thumb Down Precision =  45.70363163948059 %
Classe =  No gesture Precision =  67.68810749053955 %
Classe =  Thumb Down Precision =  99.60786700248718 %
Classe =  Thumb Down Precision =  99.98388290405273 %
Classe =  Thumb Down Precision =  91.49558544158936 %
Classe =  Thumb Down Precision =  99.39570426940918 %
Classe =  Thumb Down Precision =  97.72874116897583 %
Classe =  Thumb Down Precision =  99.21748042106628 %
Classe =  Thumb Down Precision =  99.97668862342834 %
Classe =  Sliding Two Fingers Right Precision =  99.99070167541504 %
Classe =  Sliding Two Fingers Left Precision =  69.6514368057251 %
Classe =  Sliding Two Fingers Left Precision =  99.99980926513672 %
Classe =  Sliding Two Fingers Left Preci

In [4]:
# it is for saving weights does not store the defination of model 
# new_model.save_weights('./checkpoints/3dcnn-basic')


In [10]:
#new_model.save('jester_trained_models/3dcnn-hdf5.h5')

In [11]:
new_model.save('jester_trained_models/3dcnn-basic', save_format="tf")

INFO:tensorflow:Assets written to: jester_trained_models/3dcnn-basic/assets


In [5]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --saved_model_dir jester_trained_models/3dcnn-basic  --output_dir /home/deepanshu/open_vino/udacity_project_custom_model/

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	None
	- Path for generated IR: 	/home/deepanshu/open_vino/udacity_project_custom_model/
	- IR output name: 	saved_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node names: 	None
	- Use conf